In [1]:
from qiskit_quimb import quimb_circuit
from qiskit import QuantumCircuit
import quimb as qmb
import quimb.tensor as qtn
import numpy as np

In [2]:
# Patch the quimb_circuit package, as it does not recognze 'SX' gate

from collections.abc import Sequence
from qiskit_quimb.gate import _register_gate_func
from qiskit.circuit import Instruction
from typing import Any

@_register_gate_func("sx")
def _(op: Instruction, qubits: Sequence[int], kwargs: dict[str, Any]):
    return qmb.tensor.Gate("X_1_2", params=[], qubits=qubits, **kwargs)

In [3]:
P1 = "./qasm/P1_little_peak.qasm"
P2 = "./qasm/P2_swift_rise.qasm"
P3 = "./qasm/P3__sharp_peak.qasm"
P4 = "./qasm/P4_golden_mountain.qasm"
P5 = "./qasm/P5_granite_summit.qasm"
P6 = "./qasm/P6_titan_pinnacle.qasm"

In [4]:
# Choose one circuit (not 4/5/6)
P = P3

In [5]:
try:
    qc = qtn.Circuit.from_openqasm2_file(P)
except ValueError:
    # P2 will fail due to the SX gates
    qc = quimb_circuit(QuantumCircuit.from_qasm_file(P))

In [6]:
%%time
tn = qtn.tensor_network_1d_compress(
    qc.psi, # Remove the initial state of the network
    max_bond = 32,
    cutoff = 1e-12,
    method = "fit",
    bsz = 2,  # bsz=1 is cheaper per sweep, but possibly slower to converge
    max_iterations = 100,
    tol = 1/np.sqrt(2**qc.N),
    progbar=True,
)

max_tdiff=6.32e-08:   6%|6         | 6/100 [00:05<01:33,  1.01it/s]

CPU times: user 18.1 s, sys: 16.1 s, total: 34.2 s
Wall time: 5.98 s


In [7]:
tn_mps = tn.view_as_(
    qtn.MatrixProductState,
    cyclic=False,
    L=qc.N
)

In [8]:
%%time
sampled = list(tn_mps.sample(1000))
sample_dict = {''.join(map(str, sample[0])): sample[1] for sample in sampled}

ssd = sorted(sample_dict.items(), key=lambda kv: kv[1], reverse=True)

#Reverse to get qiskit ordering
ssd[0][0][::-1],ssd[0][1]

CPU times: user 45.4 s, sys: 437 ms, total: 45.8 s
Wall time: 45.5 s


('01011000100010110011111000001010101010110001', 0.11271254171304587)